In [40]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [2]:
all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]


In [9]:
a = [True, True, True, True, True, True, False, False, False]
b = [False, False, True, True, True, True, True, True, True]
print(list(map(lambda x,y:all([x,y]), a, b)))

[False, False, True, True, True, True, False, False, False]


## 無腦買無買賣策略

In [ ]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5002, -30, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #TM
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            # allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            # allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            #T5
            allstock.loc[id, 'RS 20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 250rate>55'] = (allstock.loc[id, 'RS 250rate'] > 55)
            allstock.loc[id, 'RS 250rate<75'] = (allstock.loc[id, 'RS 250rate'] < 75)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #, 'year low sort', 'year high sort'
            #T6-
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            #T11
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate'] > 75)
            #,'RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k'
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'T5'] = all(allstock.loc[id, ['RS 20rate>85', 'RS 250rate>55', 'RS 250rate<75', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T6'] = all(allstock.loc[id, ['RS EMA250rate>80', 'RS EMA20rate>80', 'RS EMA20rate<99', 'Volume 50MA>250k']])
            allstock.loc[id, 'T11'] = all(allstock.loc[id, ['RS EMA250rate>75','RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k']])
            allstock.loc[id, 'TM'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
    T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=True).iloc[-50:].index.values.astype(str)
    T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
    TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
    
    T5_stock_num = len(T5_ID)
    T6_stock_num = len(T6_ID)
    T11_stock_num = len(T11_ID)
    TM_stock_num = len(TM_ID)
    print('刪除前  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([T5_ID,T6_ID,T11_ID,TM_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'T5', 'T6', 'T11', 'TM'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['T5','T6','T11','TM']] = False
    apexstock.loc[T5_ID,'T5'] = True
    apexstock.loc[T6_ID,'T6'] = True
    apexstock.loc[T11_ID,'T11'] = True
    apexstock.loc[TM_ID,'TM'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'T5', 'T6', 'T11', 'TM'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock = apexstock.dropna()
    print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1W = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+7))) , '%Y-%m-%d' )
    day2W = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+14))) , '%Y-%m-%d' )
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' ) 
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            except:
                try:
                    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
                except Exception as e:
                    print(e)
                    try:
                        apexstock.drop(ID, inplace=True)
                        print(f'drop stock : {ID}')
                    except:
                        pass
                    continue
            got_profit_list = ([ID in T5_ID, ID in T6_ID, ID in T11_ID, ID in TM_ID])
            # print(got_profit_list)
            try:
                initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                oneW_price = stock.loc[stock.index<day1W, 'Adj Close'].iloc[-1]
                twoW_price = stock.loc[stock.index<day2W, 'Adj Close'].iloc[-1]
                oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                twoM_price = stock.loc[stock.index<day2M, 'Adj Close'].iloc[-1]
                # print(f'day1W : {stock.loc[stock.index<day1W, "Adj Close"].index.values[-1]}')
                # print(f'day2W : {stock.loc[stock.index<day2W, "Adj Close"].index.values[-1]}')
                # print(f'day1M : {stock.loc[stock.index<day1M, "Adj Close"].index.values[-1]}')
                # print(f'day2M : {stock.loc[stock.index<day2M, "Adj Close"].index.values[-1]}')
                profit_list.append([ID, got_profit_list[0]*(oneW_price-initial_price)/initial_price,  got_profit_list[0]*(twoW_price-initial_price)/initial_price,  got_profit_list[0]*(oneM_price-initial_price)/initial_price,  got_profit_list[0]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[1]*(oneW_price-initial_price)/initial_price,  got_profit_list[1]*(twoW_price-initial_price)/initial_price,  got_profit_list[1]*(oneM_price-initial_price)/initial_price,  got_profit_list[1]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[2]*(oneW_price-initial_price)/initial_price,  got_profit_list[2]*(twoW_price-initial_price)/initial_price,  got_profit_list[2]*(oneM_price-initial_price)/initial_price,  got_profit_list[2]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[3]*(oneW_price-initial_price)/initial_price,  got_profit_list[3]*(twoW_price-initial_price)/initial_price,  got_profit_list[3]*(oneM_price-initial_price)/initial_price,  got_profit_list[3]*(twoM_price-initial_price)/initial_price])
            except Exception as e:
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    T5_stock_num = len(apexstock.loc[apexstock['T5']].index.values)
    T6_stock_num = len(apexstock.loc[apexstock['T6']].index.values)
    T11_stock_num = len(apexstock.loc[apexstock['T11']].index.values)
    TM_stock_num = len(apexstock.loc[apexstock['TM']].index.values)
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['T5 1W profit', 'T5 2W profit', 'T5 1M profit', 'T5 2M profit'
                                                   , 'T6 1W profit', 'T6 2W profit', 'T6 1M profit', 'T6 2M profit'
                                                   , 'T11 1W profit', 'T11 2W profit', 'T11 1M profit', 'T11 2M profit'
                                                   , 'TM 1W profit', 'TM 2W profit', 'TM 1M profit', 'TM 2M profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'T5 1W profit', 'T5 2W profit', 'T5 1M profit', 'T5 2M profit'
                                                   , 'T6 1W profit', 'T6 2W profit', 'T6 1M profit', 'T6 2M profit'
                                                   , 'T11 1W profit', 'T11 2W profit', 'T11 1M profit', 'T11 2M profit'
                                                   , 'TM 1W profit', 'TM 2W profit', 'TM 1M profit', 'TM 2M profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] \
    = profit_df.loc[apexstock.index.values,profit_column]
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose for all template(1Week)/' + str(day).split(' ')[0] + 'all template選股(1Week)' + '.xlsx', encoding='utf-8-sig')
    profit = {'T5 1W profit':100*profit_df.loc[apexstock.index.values,'T5 1W profit'].sum()/T5_stock_num
              ,'T5 2W profit':100*profit_df.loc[apexstock.index.values,'T5 2W profit'].sum()/T5_stock_num
              ,'T5 1M profit':100*profit_df.loc[apexstock.index.values,'T5 1M profit'].sum()/T5_stock_num
              ,'T5 2M profit':100*profit_df.loc[apexstock.index.values,'T5 2M profit'].sum()/T5_stock_num
              ,'T5 numbers of stock ':T5_stock_num
              ,'T6 1W profit':100*profit_df.loc[apexstock.index.values,'T6 1W profit'].sum()/T6_stock_num
              ,'T6 2W profit':100*profit_df.loc[apexstock.index.values,'T6 2W profit'].sum()/T6_stock_num
              ,'T6 1M profit':100*profit_df.loc[apexstock.index.values,'T6 1M profit'].sum()/T6_stock_num
              ,'T6 2M profit':100*profit_df.loc[apexstock.index.values,'T6 2M profit'].sum()/T6_stock_num
              ,'T6 numbers of stock ':T6_stock_num
              ,'T11 1W profit':100*profit_df.loc[apexstock.index.values,'T11 1W profit'].sum()/T11_stock_num
              ,'T11 2W profit':100*profit_df.loc[apexstock.index.values,'T11 2W profit'].sum()/T11_stock_num
              ,'T11 1M profit':100*profit_df.loc[apexstock.index.values,'T11 1M profit'].sum()/T11_stock_num
              ,'T11 2M profit':100*profit_df.loc[apexstock.index.values,'T11 2M profit'].sum()/T11_stock_num
              ,'T11 numbers of stock ':T11_stock_num
              ,'TM 1W profit':100*profit_df.loc[apexstock.index.values,'TM 1W profit'].sum()/TM_stock_num
              ,'TM 2W profit':100*profit_df.loc[apexstock.index.values,'TM 2W profit'].sum()/TM_stock_num
              ,'TM 1M profit':100*profit_df.loc[apexstock.index.values,'TM 1M profit'].sum()/TM_stock_num
              ,'TM 2M profit':100*profit_df.loc[apexstock.index.values,'TM 2M profit'].sum()/TM_stock_num
              ,'TM numbers of stock ':TM_stock_num}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)

In [7]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\all template(1Week).xlsx')

## 三種買賣策略

In [34]:
def strategy1_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit <-0.08:
            profit = -0.08
            break
        elif profit > 0.30:
            profit = 0.30
            break
    return profit
def strategy2_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit <-0.05:
            profit = -0.05
            break
        elif profit > 0.30:
            profit = 0.30
            break
    return profit
def strategy3_profit(stock, day, day1M):
    initial_price = stock.loc[stock.index>day, 'Adj Close']
    close_price = stock.loc[list(map(lambda x,y:all([x,y]), stock.index>day, stock.index<day1M)), 'Adj Close']
    initial_price = close_price.iloc[0]
    for i in range(len(close_price)-1):
        profit = (close_price.iloc[i+1]-initial_price)/initial_price
        if profit <-0.05:
            profit = -0.05
            break
        elif profit > 0.15:
            profit = 0.15
            break
    return profit

In [42]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5006, -36, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #TM
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            # allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            # allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            #T5
            allstock.loc[id, 'RS 20rate>85'] = (allstock.loc[id, 'RS 20rate'] > 85)
            allstock.loc[id, 'RS 250rate>55'] = (allstock.loc[id, 'RS 250rate'] > 55)
            allstock.loc[id, 'RS 250rate<75'] = (allstock.loc[id, 'RS 250rate'] < 75)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #, 'year low sort', 'year high sort'
            #T6-
            allstock.loc[id, 'RS EMA250rate>80'] = (allstock.loc[id, 'RS EMA250rate'] > 80)
            allstock.loc[id, 'RS EMA20rate>80'] = (allstock.loc[id, 'RS EMA20rate'] > 80)
            allstock.loc[id, 'RS EMA20rate<99'] = (allstock.loc[id, 'RS EMA20rate'] < 99)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            #T11
            allstock.loc[id, 'RS EMA250rate>75'] = (allstock.loc[id, 'RS EMA250rate'] > 75)
            #,'RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k'
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'T5'] = all(allstock.loc[id, ['RS 20rate>85', 'RS 250rate>55', 'RS 250rate<75', 'year low sort', 'year high sort', 'Volume 50MA>100k']])
            allstock.loc[id, 'T6'] = all(allstock.loc[id, ['RS EMA250rate>80', 'RS EMA20rate>80', 'RS EMA20rate<99', 'Volume 50MA>250k']])
            allstock.loc[id, 'T11'] = all(allstock.loc[id, ['RS EMA250rate>75','RS 20EMA is 10MAX','RS EMA20rate<99','Volume 50MA>250k']])
            allstock.loc[id, 'TM'] = all(allstock.loc[id, ['Price>150MA', 'Price>200MA', 'year high sort', 'year low sort', '200MA trending up 60d', 'RS 250rate>80', 'Volume 50MA>100k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
    T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=True).iloc[-50:].index.values.astype(str)
    T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
    TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
    
    T5_stock_num = len(T5_ID)
    T6_stock_num = len(T6_ID)
    T11_stock_num = len(T11_ID)
    TM_stock_num = len(TM_ID)
    print('刪除前  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([T5_ID,T6_ID,T11_ID,TM_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'T5', 'T6', 'T11', 'TM'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['T5','T6','T11','TM']] = False
    apexstock.loc[T5_ID,'T5'] = True
    apexstock.loc[T6_ID,'T6'] = True
    apexstock.loc[T11_ID,'T11'] = True
    apexstock.loc[TM_ID,'TM'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'T5', 'T6', 'T11', 'TM'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock = apexstock.dropna()
    print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            except:
                try:
                    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
                except Exception as e:
                    print(e)
                    try:
                        apexstock.drop(ID, inplace=True)
                        print(f'drop stock : {ID}')
                    except:
                        pass
                    continue
            got_profit_list = ([ID in T5_ID, ID in T6_ID, ID in T11_ID, ID in TM_ID])
            # print(got_profit_list)
            try:
                initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                
                profit1 = round(strategy1_profit(stock, day, day1M),3)
                profit2 = round(strategy2_profit(stock, day, day1M),3) 
                profit3 = round(strategy3_profit(stock, day, day1M),3) 
                profit4 = round((oneM_price-initial_price)/initial_price,3)
                
                profit_list.append([ID, got_profit_list[0]*profit1, got_profit_list[0]*profit2, got_profit_list[0]*profit3, got_profit_list[0]*profit4
                                    , got_profit_list[1]*profit1, got_profit_list[1]*profit2, got_profit_list[1]*profit3, got_profit_list[1]*profit4
                                    , got_profit_list[2]*profit1, got_profit_list[2]*profit2, got_profit_list[2]*profit3, got_profit_list[2]*profit4
                                    , got_profit_list[3]*profit1, got_profit_list[3]*profit2, got_profit_list[3]*profit3, got_profit_list[3]*profit4])
            except Exception as e:
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    T5_stock_num = len(apexstock.loc[apexstock['T5']].index.values)
    T6_stock_num = len(apexstock.loc[apexstock['T6']].index.values)
    T11_stock_num = len(apexstock.loc[apexstock['T11']].index.values)
    TM_stock_num = len(apexstock.loc[apexstock['TM']].index.values)
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['T5 S1 profit', 'T5 S2 profit', 'T5 S3 profit', 'T5 NS profit'
                                                   , 'T6 S1 profit', 'T6 S2 profit', 'T6 S3 profit', 'T6 NS profit'
                                                   , 'T11 S1 profit', 'T11 S2 profit', 'T11 S3 profit', 'T11 NS profit'
                                                   , 'TM S1 profit', 'TM S2 profit', 'TM S3 profit', 'TM NS profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'T5 S1 profit', 'T5 S2 profit', 'T5 S3 profit', 'T5 NS profit'
                                                   , 'T6 S1 profit', 'T6 S2 profit', 'T6 S3 profit', 'T6 NS profit'
                                                   , 'T11 S1 profit', 'T11 S2 profit', 'T11 S3 profit', 'T11 NS profit'
                                                   , 'TM S1 profit', 'TM S2 profit', 'TM S3 profit', 'TM NS profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] \
    = profit_df.loc[apexstock.index.values,profit_column]
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose for all template 3S(1Week)/' + str(day).split(' ')[0] + 'all template選股3S(1Week)' + '.xlsx', encoding='utf-8-sig')
    profit = {'T5 S1 profit':round(100*profit_df.loc[apexstock.index.values,'T5 S1 profit'].sum()/T5_stock_num,2)
              ,'T5 S2 profit':round(100*profit_df.loc[apexstock.index.values,'T5 S2 profit'].sum()/T5_stock_num,2)
              ,'T5 S3 profit':round(100*profit_df.loc[apexstock.index.values,'T5 S3 profit'].sum()/T5_stock_num,2)
              ,'T5 NS profit':round(100*profit_df.loc[apexstock.index.values,'T5 NS profit'].sum()/T5_stock_num,2)
              ,'T5 numbers of stock ':T5_stock_num
              ,'T6 S1 profit':round(100*profit_df.loc[apexstock.index.values,'T6 S1 profit'].sum()/T6_stock_num,2)
              ,'T6 S2 profit':round(100*profit_df.loc[apexstock.index.values,'T6 S2 profit'].sum()/T6_stock_num,2)
              ,'T6 S3 profit':round(100*profit_df.loc[apexstock.index.values,'T6 S3 profit'].sum()/T6_stock_num,2)
              ,'T6 NS profit':round(100*profit_df.loc[apexstock.index.values,'T6 NS profit'].sum()/T6_stock_num,2)
              ,'T6 numbers of stock ':T6_stock_num
              ,'T11 S1 profit':round(100*profit_df.loc[apexstock.index.values,'T11 S1 profit'].sum()/T11_stock_num,2)
              ,'T11 S2 profit':round(100*profit_df.loc[apexstock.index.values,'T11 S2 profit'].sum()/T11_stock_num,2)
              ,'T11 S3 profit':round(100*profit_df.loc[apexstock.index.values,'T11 S3 profit'].sum()/T11_stock_num,2)
              ,'T11 NS profit':round(100*profit_df.loc[apexstock.index.values,'T11 NS profit'].sum()/T11_stock_num,2)
              ,'T11 numbers of stock ':T11_stock_num
              ,'TM S1 profit':round(100*profit_df.loc[apexstock.index.values,'TM S1 profit'].sum()/TM_stock_num,2)
              ,'TM S2 profit':round(100*profit_df.loc[apexstock.index.values,'TM S2 profit'].sum()/TM_stock_num,2)
              ,'TM S3 profit':round(100*profit_df.loc[apexstock.index.values,'TM S3 profit'].sum()/TM_stock_num,2)
              ,'TM NS profit':round(100*profit_df.loc[apexstock.index.values,'TM NS profit'].sum()/TM_stock_num,2)
              ,'TM numbers of stock ':TM_stock_num}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    clear_output()
    print('刪除後  T5 股票數量 : ', T5_stock_num, end=' | ')
    print('T6 股票數量 : ', T6_stock_num, end=' | ')
    print('T11 股票數量 : ', T11_stock_num, end=' | ')
    print('TM 股票數量 : ', TM_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    print('day', day)
    display(all_df)

day 2017-06-21 00:00:00


,T5 S1 profit,T5 S2 profit,T5 S3 profit,T5 NS profit,T5 numbers of stock,T6 S1 profit,T6 S2 profit,T6 S3 profit,T6 NS profit,T6 numbers of stock,T11 S1 profit,T11 S2 profit,T11 S3 profit,T11 NS profit,T11 numbers of stock,TM S1 profit,TM S2 profit,TM S3 profit,TM NS profit,TM numbers of stock
2017-06-21,1.244444,1.344444,-0.322222,2.944444,9,4.766,3.198,1.330,4.980,50,1.616279,0.374419,0.639535,2.516279,43,3.205208,2.667708,1.785417,3.226042,96
2017-06-14,0.200000,0.628571,0.628571,0.342857,7,5.502,5.184,3.140,6.516,50,1.396000,2.140000,1.268000,1.500000,25,2.877108,3.390361,1.792771,3.095181,83
2017-06-07,-4.225000,-5.000000,-5.000000,-6.550000,8,-2.066,-1.610,-1.686,-2.576,50,0.397222,-0.125000,-0.200000,0.619444,36,-1.261224,-1.157143,-1.577551,-1.464286,98
2017-05-31,4.300000,4.808333,2.308333,1.766667,12,-0.954,-0.016,0.768,-1.598,50,-1.193750,-0.903125,-0.509375,-1.715625,32,2.152222,2.451111,1.651111,1.438889,90
2017-05-24,7.612500,3.987500,2.037500,6.825000,8,0.206,0.232,-0.086,-0.824,50,2.577419,2.625806,2.277419,2.735484,31,0.987368,1.205263,0.631579,1.172632,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-12-22,-2.928571,-5.000000,-5.000000,-2.771429,7,-0.002,0.952,0.644,0.414,50,-0.204545,-0.181818,0.322727,-0.159091,22,4.244231,4.238462,4.059615,4.748077,52
2009-12-15,-0.608333,0.316667,1.666667,2.233333,12,1.518,1.270,0.222,4.290,50,0.031707,-1.407317,-1.751220,3.875610,41,5.554545,3.910909,2.247273,8.676364,55
2009-12-08,-3.588889,-2.222222,-2.222222,-4.077778,9,0.878,-1.646,-1.668,0.126,50,4.618605,2.209302,2.339535,9.927907,43,5.268182,4.602273,3.963636,6.579545,44
2009-12-01,13.066667,9.266667,3.888889,20.255556,9,11.116,9.534,6.396,12.930,50,11.305128,9.058974,6.235897,12.825641,39,14.263043,12.343478,8.110870,19.767391,46


day 2017-06-28 00:00:00


In [ ]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\all template(S1-3)(1Week).xlsx')